In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
train = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
train



FileNotFoundError: ignored

train데이터 셋에서 정답칼럼을 알 수 있고, 알아야 한다. 이 데이터는 대회 제목으로 알 수 있었지만, 이 데이터에서 정답 칼럼을 구분하고 전처리를 해야한다.


In [ ]:
test = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip')
test


,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False
...,...,...,...,...
115059,45,98,2013-06-28,False
115060,45,98,2013-07-05,False
115061,45,98,2013-07-12,False
115062,45,98,2013-07-19,False


두가지 파일을 모두 불러와서 확인 한 뒤 데이터 전처리를 시작한다. 트레인 데이터셋에서 정답칼럼을 알 수 없는 경우에는 test셋과 비교하여 정답칼럼을 추측한다.
쉬운 방법은 train - test 칼럼의 차이로 알 수 있다. 여기선 Weekly_Sales 이 칼럼이 빠져있기 때문에 쉽게 알 수 있었다


In [ ]:
train_2 = train.drop(columns = ['Weekly_Sales','Date'])
train_2


,Store,Dept,IsHoliday
0,1,1,False
1,1,1,True
2,1,1,False
3,1,1,False
4,1,1,False
...,...,...,...
421565,45,98,False
421566,45,98,False
421567,45,98,False
421568,45,98,False


In [ ]:
test_2 =test.drop(columns= ['Date'])
test_2

,Store,Dept,IsHoliday
0,1,1,False
1,1,1,False
2,1,1,False
3,1,1,True
4,1,1,False
...,...,...,...
115059,45,98,False
115060,45,98,False
115061,45,98,False
115062,45,98,False


train_2 는 데이터 전처리를 위해 새롭게 만든 객체이다. 이 전처리는 1. 컴퓨터가 읽을 수 있는 숫자데이터만 남기는 처리, 2 train 과 test 셋의 칼럼을 맞추기 위한 과정이다
아래에서와 같이 test_2, train_2의 행렬이 같아진 것을 확인할 수 있다.



In [ ]:
test_2,train_2

(        Store  Dept  IsHoliday
 0           1     1      False
 1           1     1      False
 2           1     1      False
 3           1     1       True
 4           1     1      False
 ...       ...   ...        ...
 115059     45    98      False
 115060     45    98      False
 115061     45    98      False
 115062     45    98      False
 115063     45    98      False
 
 [115064 rows x 3 columns],
         Store  Dept  IsHoliday
 0           1     1      False
 1           1     1       True
 2           1     1      False
 3           1     1      False
 4           1     1      False
 ...       ...   ...        ...
 421565     45    98      False
 421566     45    98      False
 421567     45    98      False
 421568     45    98      False
 421569     45    98      False
 
 [421570 rows x 3 columns])

전처리 끝나면 모델링을 한다.
1.모델불러오기
2.학습시키기

제일 많이 사용하는 모델은 랜덤포레스트 모델이다. 그 이유는 어떤 데이터든지 사용이 쉽고, 결과 점수도 준수하게 나오기 때문이다.
랜덤포레스트 모델에는 classfier, regressor 두 가지가 있는데 이 문제에서는 regressor를 사용한다.

classfier는 분류에서 사용하며, 암치료자수, 구독자수, 고객 이탈수 예측을 위한다
regressor는 회귀문제에서 사용하며, 강수량 판매랑에서 사용한다.


In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor() #모델 선언 ()는 옵션들어감

rf.fit(train_2,train['Weekly_Sales'])
# fit()은 모델학습을 진행한다는 명령이다. 인수로는 input값과 target값이 들어간다. 지금 이 파일에선 input train_2파일이 들어갔고, 타겟값으로는 그 파일의 분석 칼럼이 들어갔다


result = rf.predict(test_2) #predict()는 fit 다음에 사용하는 함수이다.fit은 모델훈련, predict()는 예측이다.
result

array([22339.93671418, 22339.93671418, 22339.93671418, ...,
         561.02186757,   561.02186757,   561.02186757])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor() #모델 선언 ()는 옵션들어감

rf.fit(train_2,train['Weekly_Sales']) #여기서 fit이 무슨 함수인지 일단 알아야함.

result = rf.predict(test_2) #predict함수가 무엇인지 알아야
result

array([22339.93671418, 22339.93671418, 22339.93671418, ...,
         561.02186757,   561.02186757,   561.02186757])